In [1]:
import tensorflow 
from tensorflow.keras.preprocessing import image #will help to parse images
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input #preprocess_input converts input in correct format to be pass to resnet ,, see resnet is train on imagenet data , so we need to get data in same size as of imagenet 

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input
from numpy.linalg import norm

In [3]:
import pandas as pd
import numpy as np
import os
import random
from PIL import Image
from tqdm import tqdm  #tell progress of for loop
import pickle
 

In [4]:
from tensorflow.keras.applications import ResNet50

model = ResNet50(
    weights='/Users/vanshgoel/Documents/MyProject/Reverse_Image_Search(recommendsys)/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
    include_top=False,
    input_shape=(224, 224, 3)
)

model.trainable = False

In [5]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d            │ (None, 2048)           │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

None


In [6]:
#keras work on batches of image (not on single image) can do that make a batch of single image 
#ResNet50 expect input images to be exactly 224×224 pixels.
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array,axis=0) #to get a type of batches also
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img) #see we are using resnet to get features now and now we will  #here using model.predict as see already got model and we get outpus by see model predict (as model already trained)
    result.flatten()
    normalized_result = result/norm(result)

    return normalized_result  #here we got our features
"""img = image.load_img(img_path, target_size=(224, 224))
type(img)  # <class 'PIL.Image.Image'>
This is an object that stores:

Pixel data

Image format (JPEG, PNG, etc.)

Mode (e.g. 'RGB', 'L' for grayscale, 'RGBA' for transparency)"""
     

"img = image.load_img(img_path, target_size=(224, 224))\ntype(img)  # <class 'PIL.Image.Image'>\nThis is an object that stores:\n\nPixel data\n\nImage format (JPEG, PNG, etc.)\n\nMode (e.g. 'RGB', 'L' for grayscale, 'RGBA' for transparency)"

In [7]:
#will make a python list that will have have/path of all images so that we will pass it through resnet to get features/embeddings
file_names = []
for file in os.listdir('/Users/vanshgoel/Documents/MyProject/Reverse_Image_Search(recommendsys)/ImagesFashion/images'):
    file_names.append(os.path.join('ImagesFashion/images',file))

# print(len(file_names))
# print(file_names[0:5])

# for file in os.listdir("/Users/vanshgoel/Documents/MyProject/Reverse_Image_Search(recommendsys)/ImagesFashion/myntradataset/images"):
#     file_names.append(os.path.join('ImagesFashion/myntradataset/images',file))

print(len(file_names))


44441


In [8]:
random.shuffle(file_names)

In [9]:
print(len(file_names))
print(file_names[5:12])

44441
['ImagesFashion/images/21651.jpg', 'ImagesFashion/images/31458.jpg', 'ImagesFashion/images/22887.jpg', 'ImagesFashion/images/19228.jpg', 'ImagesFashion/images/55935.jpg', 'ImagesFashion/images/3986.jpg', 'ImagesFashion/images/57093.jpg']


In [10]:
n = len(file_names)//4
file_names = file_names[0:n+1]
print(len(file_names))

11111


In [11]:
has_duplicates = len(file_names) != len(set(file_names))
print(has_duplicates)   #way to check if duplicate in a list

False


In [12]:
random.shuffle(file_names)  #to do random shuffle in a list

In [13]:
# | Feature             | `PIL.Image.Image`                | `numpy.ndarray`                |
# | ------------------- | -------------------------------- | ------------------------------ |
# | Type                | Object from Pillow               | Raw array from NumPy           |
# | Data Format         | Stored as PIL image (structured) | Raw pixel values (float/int)   |
# | Common Use          | Opening, viewing, editing images | Input to ML models, pixel math |
# | Dimensions (RGB)    | (width, height)                  | (height, width, channels)      |
# | Example shape (RGB) | 224×224 (in metadata)            | `(224, 224, 3)`                |


In [14]:
# imgg = Image.open('ImagesFashion/myntradataset/images/21494.jpg')
# imgg.show()  # Opens the image using your default image viewer


In [15]:
# feature_list = []

# for file in tqdm(file_names):
#     feature_list.append(extract_features(file,model))

# print(np.array(feature_list).shape)
# print(feature_list[0:2])

In [16]:
# feature_list[0:1]


In [17]:
# import pickle
# pickle.dump(feature_list,open('embedding.pkl','wb'))
# pickle.dump(file_names,open('file_names.pkl','wb'))

In [18]:
feature_lis = pickle.load(open('embedding.pkl','rb'))
print(feature_lis)

[array([[0.        , 0.00960312, 0.        , ..., 0.00741742, 0.        ,
        0.01889571]], dtype=float32), array([[0.00207284, 0.00819871, 0.03528975, ..., 0.00048257, 0.02106146,
        0.00521342]], dtype=float32), array([[0.        , 0.01459412, 0.01682492, ..., 0.01491139, 0.03746532,
        0.02996852]], dtype=float32), array([[0.        , 0.00256785, 0.00783113, ..., 0.01373861, 0.03287283,
        0.03073308]], dtype=float32), array([[0.        , 0.02127494, 0.        , ..., 0.00370502, 0.01338724,
        0.02728689]], dtype=float32), array([[0.        , 0.        , 0.03931206, ..., 0.00445363, 0.03280219,
        0.01028161]], dtype=float32), array([[0.00196963, 0.05337647, 0.01565923, ..., 0.01085866, 0.01289043,
        0.02557419]], dtype=float32), array([[0.        , 0.00223   , 0.01287926, ..., 0.006555  , 0.03781417,
        0.05220841]], dtype=float32), array([[0.        , 0.00184183, 0.00017717, ..., 0.        , 0.00748343,
        0.01570347]], dtype=float32), 

In [19]:
for file in feature_lis:  #see right now all are [1 2048] 2d so feature_lis is 3d flat embeddings
    if file.shape != (2048,):
        print(np.array(file.shape))  


#till now 3d list will do reshape(so need to do it array first)

[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   1 2048]
[   

In [20]:
feature_array = np.array(feature_lis)
feature_array = feature_array.reshape(len(feature_array), -1)  # forces 2D


In [21]:
np.array(feature_array).shape

(11111, 2048)

In [22]:
file_nam = pickle.load(open('file_names.pkl','rb'))


In [23]:
print(file_nam)

['ImagesFashion/images/44593.jpg', 'ImagesFashion/images/7863.jpg', 'ImagesFashion/images/41309.jpg', 'ImagesFashion/images/35420.jpg', 'ImagesFashion/images/25345.jpg', 'ImagesFashion/images/46108.jpg', 'ImagesFashion/images/52026.jpg', 'ImagesFashion/images/34076.jpg', 'ImagesFashion/images/44960.jpg', 'ImagesFashion/images/43346.jpg', 'ImagesFashion/images/57577.jpg', 'ImagesFashion/images/2162.jpg', 'ImagesFashion/images/35565.jpg', 'ImagesFashion/images/33821.jpg', 'ImagesFashion/images/41722.jpg', 'ImagesFashion/images/45080.jpg', 'ImagesFashion/images/26725.jpg', 'ImagesFashion/images/18883.jpg', 'ImagesFashion/images/33391.jpg', 'ImagesFashion/images/45172.jpg', 'ImagesFashion/images/4274.jpg', 'ImagesFashion/images/7266.jpg', 'ImagesFashion/images/54126.jpg', 'ImagesFashion/images/5998.jpg', 'ImagesFashion/images/41594.jpg', 'ImagesFashion/images/52618.jpg', 'ImagesFashion/images/28005.jpg', 'ImagesFashion/images/52127.jpg', 'ImagesFashion/images/23489.jpg', 'ImagesFashion/ima

In [24]:
img_test = extract_features('test_image/1164.jpg',model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 566ms/step


In [25]:
img_test

array([[5.56593659e-06, 1.01780735e-01, 2.03068126e-02, ...,
        2.63001164e-03, 2.36919746e-02, 4.24381196e-02]], dtype=float32)

In [26]:
from sklearn.neighbors import NearestNeighbors

In [27]:
np.array(feature_array).shape

(11111, 2048)

In [28]:
neighbours = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean') #KNN need dim. < 3d
neighbours.fit(feature_array)

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=6)

In [29]:
print(np.array(img_test.shape))

[   1 2048]


In [30]:
img_test = img_test.flatten()
np.array(img_test.shape)

array([2048])

In [31]:
distances,indices  = neighbours.kneighbors([img_test])

In [32]:
print(indices)

[[  291 10098   142  4104  3321  1067]]


In [33]:
for file in indices[0]:
    print(file_nam[file])

ImagesFashion/images/3042.jpg
ImagesFashion/images/1163.jpg
ImagesFashion/images/14004.jpg
ImagesFashion/images/43054.jpg
ImagesFashion/images/2272.jpg
ImagesFashion/images/2305.jpg


In [34]:
# pip install opencv-python


In [35]:
import cv2

In [36]:
# for file in indices[0]:
#     temp_img = cv2.imread(file_nam[file])
#     cv2.imshow('output', temp_img)
#     cv2.waitKey(0)               # Waits for key press
#     cv2.destroyAllWindows()      # Closes the window *after each image*


In [37]:
def recommend(features,feature_lis):
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_lis)

    distances, indices = neighbors.kneighbors([features])

    return indices

In [38]:
def save_uploaded_file(uploaded_file):
    try:
        with open(os.path.join('uploads',uploaded_file.name),'wb') as f:
            f.write(uploaded_file.getbuffer())
        return 1
    except:
        return 0

In [39]:
import streamlit as st

In [40]:
uploaded_file = st.file_uploader("Choose an image")
if uploaded_file is not None:
    if save_uploaded_file(uploaded_file):
        # display the file
        display_image = Image.open(uploaded_file)
        st.image(display_image)
        # feature extract
        features = extract_features(os.path.join("uploads",uploaded_file.name),model)
        #st.text(features)
        # recommendention
        indices = recommend(features,feature_lis)
        # show
        col1,col2,col3,col4,col5 = st.beta_columns(5)

        with col1:
            st.image(file_nam[indices[0][0]])
        with col2:
            st.image(file_nam[indices[0][1]])
        with col3:
            st.image(file_nam[indices[0][2]])
        with col4:
            st.image(file_nam[indices[0][3]])
        with col5:
            st.image(file_nam[indices[0][4]])
    else:
        st.header("Some error occured in file upload")


2025-07-04 21:59:22.491 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-04 21:59:22.492 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-04 21:59:22.492 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-04 21:59:22.494 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-04 21:59:22.628 
  command:

    streamlit run /Users/vanshgoel/Library/Python/3.12/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-07-04 21:59:22.629 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
